In [1]:
from music21 import *

In [2]:
s = corpus.parse('bach/bwv65.2.xml')

In [3]:
s.analyze('key')

<music21.key.Key of a minor>

In [4]:
s.show('text')

{0.0} <music21.metadata.Metadata object at 0x0000014CB0845160>
{0.0} <music21.stream.Part Soprano>
    {0.0} <music21.instrument.Instrument P1: Soprano: Instrument 1>
    {0.0} <music21.stream.Measure 0 offset=0.0>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.Key of C major>
        {0.0} <music21.meter.TimeSignature 3/4>
        {0.0} <music21.note.Note A>
    {1.0} <music21.stream.Measure 1 offset=1.0>
        {0.0} <music21.note.Note A>
        {1.0} <music21.note.Note A>
        {2.0} <music21.note.Note A>
    {4.0} <music21.stream.Measure 2 offset=4.0>
        {0.0} <music21.note.Note B>
        {2.0} <music21.note.Note B>
    {7.0} <music21.stream.Measure 3 offset=7.0>
        {0.0} <music21.note.Note C>
        {1.0} <music21.note.Note B>
        {2.0} <music21.note.Note A>
    {10.0} <music21.stream.Measure 4 offset=10.0>
        {0.0} <music21.note.Note G>
    {13.0} <music21.stream.Measure 5 offset=13.0>
        {0.0} <music21.note.Note C>
        {2.0} 

In [31]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [33]:
part_one_notes = flatten([m.notes for m in s.parts[0].measures(0, None) if m.isStream])

In [37]:
print(part_one_notes[0].pitch)
print(part_one_notes[0].duration)

A4
<music21.duration.Duration 1.0>
